In [1]:
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from datetime import datetime
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
processed_data=pd.read_json("../data/processed_train.json")

In [2]:
random.seed(0)
train_data=processed_data.sample(n=processed_data.shape[0]*7/10)
test_data=processed_data.drop(train_data.index)

In [3]:
train_data.columns

Index([u'By Owner', u'Cats Allowed', u'Common Outdoor Space', u'Dogs Allowed',
       u'Doorman', u'Elevator', u'Exclusive', u'Fitness Center', u'Furnished',
       u'Laundry In Building', u'Laundry In Unit', u'No Fee', u'Parking Space',
       u'Private Outdoor Space', u'Reduced Fee', u'Short Term Allowed',
       u'Storage Facility', u'Sublet / Lease-Break', u'bathrooms', u'bedrooms',
       u'building_id', u'created', u'description', u'display_address',
       u'features', u'interest_level', u'latitude', u'listing_id',
       u'longitude', u'manager count', u'manager_id', u'photos', u'price',
       u'street_address', u'unique_count'],
      dtype='object')

In [4]:
##########################first try with all numeric features in datasets##########################################
train=train_data.drop(['building_id','created','description','display_address','manager_id','longitude','latitude','listing_id','photos','street_address','features'],axis=1)
test=test_data.drop(['building_id','created','description','display_address','manager_id','longitude','latitude','listing_id','photos','street_address','features'],axis=1)

In [5]:
y_train=train.loc[:,'interest_level']
x_train=train.drop('interest_level',axis=1)
y_test=test.loc[:,'interest_level']
x_test=test.drop('interest_level',axis=1)

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = DecisionTreeClassifier(max_depth=4)
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('dt',clf4)], voting='soft')

params = {'lr__C': [100, 1e4], 'rf__n_estimators': [50, 200],}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(x_train, y_train)

In [14]:
sum(grid.predict(x_train)==y_train)*1.0/y_train.shape[0]

0.8711283506049905

In [15]:
sum(grid.predict(x_test)==y_test)*1.0/y_test.shape[0]

0.70849655545049306

In [10]:
###using Logistic Reg, Random Forest, GaussianNB, DecisionTreeClassifier(bagging), KNN, SVC###
